# M5 Forecasting - Accuracy

Соревнование на платформе Kaggle для предсказания продаж по различным продуктам сети Walmart на 28 дней вперед.

Набор данных включает в себя данный о продажах 3049 продуктов, классифицированных на 3 категории: Хобби, Продукты питания и Товары для дома, и разделенных еще на 7 категорий.
Продукты продаются в 10 магазинах 3 штатов (Калифорния, Техас и Вайоминг). 

## Файлы

* <code>calendar.csv</code> Содержит информацию о датах продажи товаров.

* <code>sales_train_validation.csv</code> Содержит ежедневные данные о продажах товаров в каждом магазине 

* <code>sample_submission.csv</code> Пример предоставления данных.

* <code>sell_prices.csv</code> Содержит информацию о ценах по дням.

Файл 'calerndar.csv' включает информацию о датах продаж:
* <code>date</code> - Дата в формате 'y-m-d'
* <code>wm_yr_wk</code> - Идентификатор недели, к которому относится дата
* <code>weekday</code> - День недели (понедельник, вторник, ...)
* <code>wday</code> - Идентификатор дня недели,начиная с воскресенья
* <code>month</code> - Месяц
* <code>year</code> - Год
* <code>d</code> - Номер дня
* <code>event_name_1</code> - Название праздника
* <code>event_type_1</code> - Тип праздника
* <code>event_name_2</code> - Название праздника,  если он не вошел в event_name_1
* <code>event_type_2</code> - Тип праздника, если он не вошел в event_name_1
* <code>snap_CA, snap_TX, snap_WI</code> - Бинарная переменная (0, 1), указывющая, осуществляются ли в указанную дату в магазинах Калифорнии, Техаса и Вайоминга покупки SNAP. SNAP - продовольственная программа правительства США по выдаче бесплатных продуктов 

Файл 'sales_train_validation.csv' содержит информацию о ежедневных данных о продажах на единицу товара и магазина:
* <code>id</code> - Идентификатор 
* <code>item_id</code> - Идентификатор продукта
* <code>dept_id</code> - Идентификатор отдела, к которому относится продукт
* <code>cat_id</code> - Идентификатор категории продукта
* <code>store_id</code> - Идентификатор магазина
* <code>state_id</code> - Штат, в котором расположен магазин
* <code>d_1, d_2, ... , d_1914</code> - Количество покупок в день, начиная с даты 29.01.2011

Файл 'sell_price.csv' содержит информацию о ежедневных ценах на товар:
* <code>store_id</code> - Идентификатор магазина
* <code>item_id</code> - Идентификатор продукта
* <code>wm_yr_wk</code> - Идентификатор недели
* <code>sell_price</code> - Цена продукта

In [25]:
import pandas as pd
import numpy as np
from  datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
import pickle
import random

In [ ]:
pd.options.display.max_columns = 50 

In [2]:
calendar_dtypes = {'event_name_1': 'category', 
                   'event_name_2': 'category',
                   'event_type_1': 'category', 
                   'event_type_2': 'category',
                   'weekday': 'category', 
                   'wm_yr_wk': 'int16', 
                   'wday': 'int16',
                   'month': 'int16',
                   'year': 'int16',
                   'snap_CA': 'int16', 
                   'snap_TX': 'int16',
                   'snap_WI': 'int16'}

sell_price_dtypes = {'store_id': 'category',
                     'item_id': 'category',
                     'wm_yr_wk': 'int16',
                     'sell_price': 'float32'}

In [3]:
# функция преобразовывает категориальные переменные в целые числа
def encoding(df, dictionary):
    for column, column_type in dictionary.items():
        if column_type == 'category':
            df[column] = df[column].cat.codes.astype("int16")
            df[column] -= df[column].min()
        
    return df

In [4]:
sell_prices = pd.read_csv(r'sell_prices.csv', dtype=sell_price_dtypes)
encoding(sell_prices, sell_price_dtypes)
print('Sell_prices содержит {} строк и {} колонок'.format(sell_prices.shape[0], sell_prices.shape[1]))

,store_id,item_id,wm_yr_wk,sell_price
0,0,0,11325,9.58
1,0,0,11326,9.58
2,0,0,11327,8.26
3,0,0,11328,8.26
4,0,0,11329,8.26
...,...,...,...,...
6841116,9,3048,11617,1.00
6841117,9,3048,11618,1.00
6841118,9,3048,11619,1.00
6841119,9,3048,11620,1.00


In [5]:
calendar = pd.read_csv(r'calendar.csv', dtype=calendar_dtypes)
calendar['date'] = pd.to_datetime(calendar['date'])
encoding(calendar, calendar_dtypes)
print('Calendar содержит {} строк и {} колонок'.format(calendar.shape[0], calendar.shape[1]))

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,6,5,6,2016,d_1965,0,0,0,0,0,1,1
1965,2016-06-16,11620,4,6,6,2016,d_1966,0,0,0,0,0,0,0
1966,2016-06-17,11620,0,7,6,2016,d_1967,0,0,0,0,0,0,0
1967,2016-06-18,11621,2,1,6,2016,d_1968,0,0,0,0,0,0,0


In [6]:
categorical_colunms = ['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id']
num_columns = [f"d_{day}" for day in range(1,1941+1)]
stv_dtypes = {num_col:"float32" for num_col in num_columns} 
stv_dtypes.update({column: "category" for column in categorical_colunms})

sales_train_validation = pd.read_csv(r'sales_train_validation.csv', dtype=stv_dtypes)
print('Sales_train_validation содержит {} строк и {} колонок'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))

for column in categorical_colunms:
    sales_train_validation[column] = sales_train_validation[column].cat.codes.astype("int16")
    sales_train_validation[column] -= sales_train_validation[column].min()
    
id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
product = sales_train_validation[id_columns].drop_duplicates()

In [ ]:
submission = pd.read_csv(r'sample_submission.csv')
validate_submission = submission[submission.id.str.endswith('validation')]
eval_submission = submission[submission.id.str.endswith('evaluation')]

newcolumns = ["id"] + ["d_{}".format(i) for i in range(1914, 1914+28)]
validate_submission.columns = newcolumns
validate_submission = validate_submission.merge(sell_prices, how = 'left', on = 'id')

In [7]:
# создание датафрейма, объединяющего данные из всех датафреймов
df = pd.melt(sales_train_validation, 
             id_vars = ['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id'], 
             value_vars = [column for column in sales_train_validation.columns if column.startswith("d_")], 
             var_name = "d", 
             value_name = "sales")

In [8]:
df = df.merge(calendar, on= "d", copy = False)
df = df.merge(sell_prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [10]:
# создание фичей
# дабоваление столбцов со скользящими значениями по неделям и месяцам

lags = [7, 28] 
lag_columns = [f'lag_{lag}' for lag in lags]
for lag, lag_column in zip(lags, lag_columns):
    df[lag_column] = df[['id', 'sales']].groupby('id')['sales'].shift(lag)

windows = [7, 28]
for window in windows :
    for lag, lag_column in zip(lags, lag_columns):
        df[f'rmean_{lag}_{window}'] = df[['id', lag_column]].groupby('id')[lag_column]\
                                      .transform(lambda x : x.rolling(window).mean())

In [11]:
date_features = {'wday': 'weekday',
                 'week': 'weekofyear',
                 'month': 'month',
                 'quarter': 'quarter',
                 'year': 'year',
                 'mday': 'day'}
    
for date_name, date_func in date_features.items():
    if date_name in df.columns:
        df[date_name] = df[date_name].astype("int16")
    else:
        df[date_name] = getattr(df['date'].dt, date_func).astype("int16")

In [12]:
df.dropna(inplace=True)
print('Итоговый датафрейм содержит {} строк и {} колонок'.format(df.shape[0], df.shape[1]))

In [13]:
X = df.drop(['id', 'date', 'sales', 'd', 'wm_yr_wk', 'weekday'], axis=1)
y = df['sales']

In [14]:
del df, calendar, sell_prices, sales_train_validation; gc.collect()

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
categorical_features = ['item_id', 'dept_id', 'store_id',
             'cat_id', 'state_id', 'event_name_1',
             'event_name_2', 'event_type_1', 'event_type_2']
             
train_set = lgb.Dataset(X_train, y_train)
val_set = lgb.Dataset(X_val, y_val)
del X_train, X_val, y_train, y_val; gc.collect()

In [21]:
parameters = {'objective': 'poisson',
              'force_row_wise': True,
              'learning_rate': 0.075,
              'sub_row': 0.75,
              'bagging_freq': 1,
              'lambda_l2': 0.1,
              'metric': ['rmse'],
              'verbosity': 1,
              'num_iterations' : 3000,
              'num_leaves': 128,
              'min_data_in_leaf': 100,}

In [ ]:
import pickle 
dbfile = open('train_set.pkl', 'wb') 
pickle.dump(train_set, dbfile)
dbfile.close() 

dbfile = open('val_set.pkl', 'wb') 
pickle.dump(val_set, dbfile)
dbfile.close() 

In [ ]:
model = lgb.train(parameters, train_set, valid_sets = [test_set], verbose_eval = 50)

In [ ]:
model.save_model("model.lgb")

In [ ]:
import pickle 
dbfile = open('LightGBM.pkl', 'wb') 
pickle.dump(model, dbfile)
dbfile.close() 

In [34]:
test_num_columns = [f'd_{day}' for day in range(1913-86,1914)]
test_categorical_columns = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
dtype = {column: 'float32' for column in test_num_columns} 
dtype.update({column: 'category' for column in test_categorical_columns if column != 'id'})
test = pd.read_csv('sales_train_validation.csv', 
                   nrows = None, 
                   usecols = test_categorical_columns + test_num_columns,
                   dtype = dtype)
    
for column in test_categorical_columns:
    if column != 'id':
        test[column] = test[column].cat.codes.astype('int16')
        test[column] -= test[column].min()
    

for day in range(1914, 1943):
    test[f"d_{day}"] = np.nan
    
test = pd.melt(test, 
               id_vars = test_categorical_columns,
               value_vars = [col for col in test.columns if col.startswith('d_')],
               var_name = 'd',
               value_name = 'sales')
    
test = test.merge(calendar, on= 'd', copy = False)
test = test.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], copy = False)

In [35]:
F_columns = [f"F{i}" for i in range(1,29)]

In [ ]:
lags = [7, 28] 
lag_columns = [f'lag_{lag}' for lag in lags]
for lag, lag_column in zip(lags, lag_columns):
    test[lag_column] = test[['id', 'sales']].groupby('id')['sales'].shift(lag)

windows = [7, 28]
for window in windows :
    for lag, lag_column in zip(lags, lag_columns):
        test[f'rmean_{lag}_{window}'] = test[['id', lag_column]].groupby('id')[lag_column]\
                                      .transform(lambda x : x.rolling(window).mean())

In [ ]:
def predict(model, X_train, X_val, factor=1):
    DATES = X_test["date"].unique()
    NDATE = len(DATES)
    print("NDATE", NDATE)
    
    col = ["id"] + ["F{}".format(i) for i in range(1, NDATE+1)]
    itemId = X_train["dept_id"].unique()
    print("#CHUNK", len(itemId))
    
    acc_o = []
    itemId = sorted(itemId)
    for iid in itemId:
        test = X_test[X_test["dept_id"]==iid]
        
        ids = test["id"].unique()
        oarr = np.zeros((len(ids), NDATE+1))
        o = pd.DataFrame(oarr, columns=col)
        
        o["id"] = test[test["date"]==DATES[0]]["id"].values
        
        train = X_train[X_train["dept_id"]==iid]
        
        ## XX=test, X=train
        lastmonth = pd.to_datetime(train.head(1)["date"])
        for idx, date in enumerate(DATES):
            
            newrow = test[test["date"]==date]
            train = train.append(newrow)
            
            train.sort_values(by=['id', "date"], inplace=True)
# #             print("num feats START")
            feat = numerical_feature(train)
            
# #             print("num feats DONE")

#             print(f"============== {idx} ==========")
#             p = feat[feat["id"]=="FOODS_1_001_CA_1_validation"]
# #             print(p)
#             print(p.tail(15)[["date", "demand", "shifted_t7"]])
#             if idx==10:
#                 return None
            
            x = feat.loc[feat["date"] == date , train_cols]
            val_pred = model.predict(x, num_iteration=model.best_iteration)

            
            o[f"F{idx+1}"] = val_pred*factor
            
            
            train.loc[train["date"]==date, "demand"] = val_pred*factor
            
            
            lastmonth = lastmonth + pd.DateOffset(days=1)
            train = train[train['date'] >= str(lastmonth.values[0])]
            
        acc_o.append(o)
        acc_o = [pd.concat(acc_o)]
        print(iid)
#         break
    
    acc_o = pd.concat(acc_o)
    return acc_o